In [63]:
#Import and declare TIC

%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightkurve as lk
from scipy.interpolate import make_interp_spline as spline
import scipy.signal as signal
import matplotlib

plt.style.use('seaborn-v0_8-darkgrid')

TIC = 'TIC 413482626'

In [64]:
#Download the data and stitch it together

search_results = lk.search_lightcurve(TIC, cadence='short', author='SPOC')
print(search_results)

#Enter the data index to be used                                                                                        [INPUT NEEDED]
use_till = 30

lcs = []
for i in range(use_till):
    try:
        lcs.append(search_results[i].download(quality_bitmask='hardest'))
    except:
        pass


No data found for target "TIC 413482626".


SearchResult containing 0 data products.


In [65]:
#Applying the savgol filter and plotting (This part defines the function AND removes the big features)

def apply_savgol_filter(time, flux, window_length_for_remove : int = 7500, mode : str = 'remove', window_length_for_gaussian : int = 100, polyorder = 4, displaygraphs : bool = True) -> pd.DataFrame:
    '''
    Applies the savgol filter to `flux` and `time`.

    Works in two modes:
    - `'remove'`: Removes the outburst and other large features
    - `'gaussian'`: Finds the gaussians, techincally

    Parameters:
    ----------
    time : array-like
        The time array of the lightcurve
    flux : array-like
        The flux array of the lightcurve
    mode : str
        The mode in which the function should work, either `'remove'` or `'gaussian'`
    window_length_for_gaussian : int
        The window length for the gaussian filter, works only in `'gaussian'` mode
    window_length_for_remove : int
        The window length for the remove filter, works only in `'remove'` mode
    polyorder : int
        The polynomial order for the savgol filter

    Returns:
    ----------
    if `mode == 'remove'`:
        returns the corrected lightcurve as a pandas.DataFrame object
        Columns: `'time'` and  `'flux'`
    if `mode == 'gaussian'`:
        returns the gaussians as a pandas.DataFrame object
        Columns: `'time'` and  `'gaussian_flux'`

    '''
    if mode == 'remove':  #Removing the outburst and other large features
        flx = signal.savgol_filter(flux, int(window_length_for_remove), polyorder)
        flx2 = signal.savgol_filter(flx, int(window_length_for_remove), polyorder)

        if displaygraphs:
            #plotting the savgol
            plt.figure(figsize=(10, 5))
            plt.title('Applying SavGol to remove large features')
            plt.plot(time, flux, 'r-', lw=0.5, label='Raw Light Curve')
            plt.plot(time, flx, 'k-', lw=1, label='Initial Fit')
            plt.plot(time, flx2, 'b-', lw=1, label='Final Fit')
            plt.legend()

            #plotting the processed graph
            plt.figure(figsize=(10, 5))
            plt.title('Corrected Light Curve')
            plt.plot(time, flux - flx2, 'b-', lw=0.5)

        #Building the lightcurve again, with the correction
        return pd.DataFrame({
            'time':time,
            'flux':flux - flx2
        }, index=None)

    elif mode == 'gaussian':   #Finding the gaussians, techincally
        fit_flux = signal.savgol_filter(flux, int(window_length_for_gaussian), polyorder)

        if displaygraphs:
            plt.figure(figsize=(10, 5))
            plt.title('Gaussian Fitting')
            plt.plot(time, flux, 'r-', lw=0.5, label='Corrected Light Curve')
            plt.plot(time, fit_flux, 'k-', lw=1, label='Fitted Gaussians')
            plt.legend()

        return pd.DataFrame({
            'time':time,
            'gaussian_flux': - fit_flux
        }, index=None)
    
    elif mode == 'gaussian_twice':   #Finding the gaussians, techincally
        fit_flux = signal.savgol_filter(flux, int(window_length_for_gaussian), polyorder)
        fit_fluxx = signal.savgol_filter(fit_flux, 50, polyorder)
        if displaygraphs:
            plt.figure(figsize=(10, 5))
            plt.title('Gaussian Fitting')
            plt.plot(time, flux, 'r-', lw=0.5, label='Corrected Light Curve')
            plt.plot(time, fit_flux, 'k-', lw=1, label='Fitted Gaussians')
            plt.plot(time, fit_fluxx, 'b-', lw=1, label='Fitted Gaussians Final?')
            plt.legend()

        return pd.DataFrame({
            'time':time,
            'gaussian_flux': - fit_fluxx
        }, index=None)

#Function for checking the differences in peaks
def process_gaussians(fitted : lk.lightcurve.LightCurve | pd.DataFrame, threshold):
    '''
    Finds the peaks, the periods and mean of the periods for fitted gaussians curve.

    Parameters:
    -----------
    `fitted` : `lightkurve.LightCurve` or `pd.Dataframe` with columns `'time'` and `'gaussian_flux'` 
        The lightcurve that has been fitted for gaussians.
    `threshold` : `float`
        The threshold for the peak detection.
    `number_of_gaps` : `int`
        (default = 10)
        The aproximate number of gaps, always enter a much higher value than the true number.

    Returns:
    --------
    mean_diff : `float`
        The mean difference between the peaks.
    peaks_n_periods : `pd.DataFrame`
        Columns:
        - 'period' : The periods between the peaks
        - 'time' : The peak times. If is period is `p2 - p1`, this returns `p2`.
    '''

    import lightkurve as lk
    import pandas as pd
    from scipy.signal import find_peaks
    import numpy as np
    import matplotlib.pyplot as plt

    # Get the flux and time values
    if isinstance(fitted, lk.LightCurve):
        fl = np.array(fitted.flux)
        tm = np.array(fitted.time.btjd)
    if isinstance(fitted, pd.DataFrame):
        fl = np.array(fitted['gaussian_flux'])
        tm = np.array(fitted['time'])

    # Find peaks in fl above threshold
    peaks, _ = find_peaks(fl, height=threshold)

    # Get the associated time values for the peaks
    peak_times = tm[peaks]

    # mean_diff = np.sort(np.diff(peak_times))[: - number_of_gaps].mean()

    return pd.DataFrame({
        'period' : np.diff(peak_times),
        'time' : peak_times[1:]
    })

#Functions for rejecting outliers
def reject_outliers(data, m=1):
    removed = data[abs(data - np.mean(data)) > m * np.std(data)]
    print("Outliers Removed: ", len(removed))
    accepted = data[abs(data - np.mean(data)) < m * np.std(data)]
    plt.figure()
    plt.plot(accepted)
    return accepted
def reject_outliers_pd(data : pd.DataFrame, column_name : str, m=1):
    '''
    Removes the outliers from the data.

    Parameters:
    ----------
    data : pd.DataFrame
        The data from which the outliers are to be removed.
    column_name : str
        The column name from which the outliers are to be removed.
    m : int
        The number of standard deviations to be considered as an outlier.
    
    Returns:
    --------
    accepted : pd.DataFrame
        The data with the outliers removed.
    Prints the number of outliers removed.
    
    '''
    removed = data[abs(data[column_name] - np.mean(data[column_name])) > m * np.std(data[column_name])]
    print("Outliers Removed: ", len(removed))
    accepted = data[abs(data[column_name] - np.mean(data[column_name])) < m * np.std(data[column_name])]
    accepted.reset_index(inplace=True, drop=True)
    return accepted

#Function for making the O-C diagram
def make_OC_diagram(accepted : pd.DataFrame, calculate_from : int = 1):
    '''
    Makes the O-C diagram from the accepted data.

    Parameters:
    ----------
    accepted : pd.DataFrame
        The data from which the O-C diagram is to be made. Must have columns 'time' and 'period'.
    calculate_from : int
        (default = 1)
        The number of periods to calculate the CALCULATED period from.
    
    Returns:
    --------
    OC_DataFrame : pd.DataFrame
        The O-C diagram data.
        Columns:
        - 'T' : Time of the period from the start
        - 'E' : Event Number
        - 'O-C' : The O-C values
    '''
    df = accepted.copy(deep=True)

    df['T'] = df['time'] - df['time'][0]
    p0 = df['period'][:calculate_from].mean()
    df['p_'] = [ ( df['period'][:x].sum() / x ) for x in range(1, len(df) + 1)]
    df['E'] = df['T'] / df['p_']
    df['E'] = df['E'].astype(int)
    df['O-C'] = df['T'] * ( 1 - ( p0 / df['p_'] ) )
    return df[['T', 'E', 'O-C']]


In [66]:
window_length_for_remove = 7500        #The window length for the remove filter                        [INPUT NEEDED]
window_length_for_gaussian = 200       #The window length for the gaussian filter                      [INPUT NEEDED]

show_graphs = True                    #Show the graphs or not                                         [INPUT NEEDED]

m = 0.45                               #Number of standard deviations to be considered as an outlier   [INPUT NEEDED]
threshold = 15                         #Threshold for the peak detection                               [INPUT NEEDED]
how_many_times_to_remove_outliers = 2  #Number of times to remove outliers                             [INPUT NEEDED]

In [67]:
time_interval = 1

times = [np.array(x.time.btjd) for x in lcs]
cadence_multiplier = 86400/(np.median(np.diff(times[0][0:100])) * 86400).round()
olds = [x[0] for x in times]
times = [(x - x[0]) * cadence_multiplier for x in times]
times = [x.round(0) for x in times]
fluxes = [np.array(x.flux, dtype=float) for x in lcs]

min_times = [x[0] for x in times]
max_times = [x[-1] for x in times]
full_times = [np.arange(x, y + time_interval, time_interval) for x, y in zip(min_times, max_times)]
savgols = []

for i in range(len(times)):
    #Filling in missing times with nans, and then interpolating gaps with staright lines
    df = pd.DataFrame({'Time':times[i], 'Flux':fluxes[i]})
    df.set_index('Time', inplace=True)
    df = df.reindex(full_times[i])
    df['Flux'] = df['Flux'].interpolate(method='linear')
    df.reset_index(inplace=True)
    df.rename(columns={'index':'Time'}, inplace=True)
    df['Time'] = (df['Time'] / cadence_multiplier) + olds[i] + 2457000
    df.dropna(inplace=True)
    df.reset_index(inplace=True)

    #Interpolating using spline
    time_smooth = np.linspace(df['Time'].min(), df['Time'].max(), len(df['Time']) * 4)
    spl = spline(df['Time'], df['Flux'], k=3)
    flux_smooth = spl(time_smooth)

    #Applying the savgol filter
    del df
    df = pd.DataFrame({'time':time_smooth, 'flux':flux_smooth})
    df = apply_savgol_filter(df['time'], df['flux'], mode='remove', window_length_for_remove=window_length_for_remove, displaygraphs=show_graphs)
    savgols.append(df)



IndexError: list index out of range

In [ ]:
combined_df = pd.concat(savgols, ignore_index=True)
combined_df.info()
combined_df.plot('time', 'flux', lw=0.4)

gaussian_df = apply_savgol_filter(combined_df['time'], combined_df['flux'], mode = 'gaussian_twice', window_length_for_gaussian = window_length_for_gaussian)

gaussian_df.plot('time', 'gaussian_flux', lw=0.4)


In [ ]:
#Finding the peaks and periods
times_n_periods = process_gaussians(gaussian_df, threshold)
accepted_times_n_periods = times_n_periods.copy(deep=True)

for i in range(how_many_times_to_remove_outliers):
    accepted_times_n_periods = reject_outliers_pd(accepted_times_n_periods, 'period', m)

#Plotting the periods
plt.figure()
plt.title('Periods')
plt.plot(accepted_times_n_periods['time'], accepted_times_n_periods['period'], color = 'orange', marker = '.', markersize = 4, lw = 0, label = 'Accepted Periods')
plt.plot(times_n_periods['time'], times_n_periods['period'], color = 'blue', marker = '.', markersize = 1.5, lw = 0, label = 'All Periods')
# plt.plot(times_n_periods['time'], times_n_periods['period'].mean(), color = 'red', lw = 1, label = 'Mean Period')
plt.xlabel('Time')
plt.ylabel('Period')

In [ ]:
#Function for making OC diagram
OC_DataFrame = make_OC_diagram(accepted_times_n_periods, calculate_from=1)

#Plotting the O-C diagram
plt.figure(figsize=(10, 5))
plt.title('O-C Diagram')
plt.plot(OC_DataFrame['E'], OC_DataFrame['O-C'], color = 'green', marker = '.', markersize = 4, lw = 0.5, label = 'O-C Diagram')
plt.xlabel('Event Number')
plt.ylabel('O-C')
plt.show()